# 第7章　航空公司客户价值分析

## 任务7-2　预处理航空客户数据

<center>
    <br>
    代码7-1　缺失值与异常值处理
</center>

In [1]:
import numpy
import pandas
# 导入数据
airline_data = pandas.read_csv('data/air_data.csv', encoding='GB18030')
print('原始数据的形状为：', airline_data.shape)
# 去除票价为空的记录
expression_A = airline_data['SUM_YR_1'].notnull()
expression_B = airline_data['SUM_YR_2'].notnull()
expression = expression_A & expression_B
airline_notNull = airline_data.loc[expression, :]
print('删除缺失记录后数据的形状为：', airline_notNull.shape)

原始数据的形状为： (62988, 44)
删除缺失记录后数据的形状为： (62299, 44)


In [2]:
index_A = airline_notNull['SUM_YR_1'] != 0
index_B = airline_notNull['SUM_YR_2'] != 0
index_C = (airline_notNull['SEG_KM_SUM'] > 0) & (airline_notNull['avg_discount'] != 0)
airline = airline_notNull[(index_A | index_B) & index_C]
print('删除异常记录后数据的形状为：', airline.shape)

删除异常记录后数据的形状为： (62044, 44)


<center>
    <br>
    代码7-2　选取并构建<code>LRFMC</code>模型的特征
</center>

In [3]:
# 选取需求特征
airline_selection = airline[[
    'FFP_DATE', 'LOAD_TIME', 'FLIGHT_COUNT',
    'LAST_TO_END', 'avg_discount', 'SEG_KM_SUM'
]]
# 构建 L 特征
L = (
    pandas.to_datetime(airline_selection['LOAD_TIME'])
    - pandas.to_datetime(airline_selection['FFP_DATE'])
)
L = L.astype('str').str.split().str[0]
L = L.astype('int') / 30
# 合并特征
airline_features = pandas.concat([L, airline_selection.iloc[:, 2:]], axis=1)
print('构建的 LRFMC 特征前5行为：', airline_features.head(), sep='\n')

构建的 LRFMC 特征前5行为：
           0  FLIGHT_COUNT  LAST_TO_END  avg_discount  SEG_KM_SUM
0  90.200000           210            1      0.961639      580717
1  86.566667           140            7      1.252314      293678
2  87.166667           135           11      1.254676      283712
3  68.233333            23           97      1.090870      281336
4  60.533333           152            5      0.970658      309928


<center>
    <br>
    代码7-3　标准化<code>LRFMC</code>模型的特征
</center>

In [4]:
from sklearn.preprocessing import StandardScaler
data = StandardScaler().fit_transform(airline_features)
numpy.savez('stash/airline_scale.npz', data)
print('标准化后 LRFMC 五个特征的前5行为：', data[:5, :], sep='\n')

标准化后 LRFMC 五个特征的前5行为：
[[ 1.43571897 14.03412875 -0.94495516  1.29555058 26.76136996]
 [ 1.30716214  9.07328567 -0.9119018   2.86819902 13.1269701 ]
 [ 1.32839171  8.71893974 -0.88986623  2.88097321 12.65358345]
 [ 0.65848092  0.78159082 -0.41610151  1.99472974 12.54072306]
 [ 0.38603481  9.92371591 -0.92291959  1.3443455  13.89884778]]
